In [ ]:
import pandas as pd 
purchase_df = pd.read_csv("/kaggle/input/purchase-behavior-dataset/purchase behaviou dataset.csv")
transaction_df = pd.read_excel("/kaggle/input/quantium/QVI_transaction_data.xlsx")

In [ ]:
purchase_df.head()

In [ ]:
transaction_df.head()

In [ ]:
purchase_df['LIFESTAGE'].isnull().sum()

In [ ]:
purchase_df['LYLTY_CARD_NBR'].isnull().sum()

In [ ]:
purchase_df['PREMIUM_CUSTOMER'].isnull().sum()

Hence, there are no missing values in any of the columns in purchase_df. Now checking for any duplicate values. 

In [ ]:
purchase_df['LIFESTAGE'].unique()

In [ ]:
purchase_df['PREMIUM_CUSTOMER'].unique()

In [ ]:
duplicate_rows = purchase_df[purchase_df.duplicated()]
print(duplicate_rows)

There are no duplicate records in the dataframe. 

In [ ]:
duplicate_rows = purchase_df[purchase_df.duplicated(['LYLTY_CARD_NBR'])]
print(duplicate_rows)

There are no duplicate entries for Loyalty Card Number as well. The dataframe purchase_df is good to go! 


Now moving onto transaction_df to check for similar checks 

In [ ]:
transaction_df['LYLTY_CARD_NBR'].isnull().sum()

Looking for duplicate enteries in the dataframe 

In [ ]:
duplicate_rows = transaction_df[transaction_df.duplicated()]
print(duplicate_rows)

In [ ]:
duplicate_rows = transaction_df[transaction_df["LYLTY_CARD_NBR"] == 107024]
print(duplicate_rows)

Since there is an record for which duplicates exist, we will drop the duplicate entries, only keeping the first one. 

In [ ]:
transaction_df = transaction_df.drop_duplicates()

In [ ]:
duplicate_rows = transaction_df[transaction_df.duplicated()]
print(duplicate_rows)

Checking for the datatype of columns in both dataframes 


In [ ]:
purchase_df.dtypes

In [ ]:
transaction_df.dtypes

In [ ]:
#transaction_df['DATE'] = pd.to_datetime(transaction_df['DATE'])
#print(transaction_df)

Merging the two dataframes together now based on TXN_ID. 

In [ ]:
merged_df = pd.merge(purchase_df, transaction_df, on = ['LYLTY_CARD_NBR'])
merged_df.shape

In [ ]:
merged_df.head(5)

Since we are concerned with chips products, taking a look at all the products to determine the chip products in them.

In [ ]:
merged_df['PROD_NAME'].unique()

In [ ]:
chips_df = merged_df[merged_df['PROD_NAME'].str.contains(" Chips  | chip  ")]
chips_df

In [ ]:
chips_df.head(10)

DRopping the date column as it is invalid data. The date doesnt follow any formats(YYYYMMDD/DDMMYYYY)

In [ ]:
del chips_df['DATE']
chips_df.shape

Doing the analysis to see the pattern for buying chips among customers. 

In [ ]:
lifestage_df = chips_df.groupby('LIFESTAGE')['PROD_QTY'].sum()
lifestage_df

In [ ]:
import plotly.express as px 
fig = px.bar(lifestage_df, orientation = 'h')
fig.show()

In [ ]:
premium_customer_df = chips_df.groupby('PREMIUM_CUSTOMER')['PROD_QTY'].sum()
premium_customer_df

In [ ]:
fig2 = px.bar(premium_customer_df, orientation = 'v')
fig2.show()

Combining these two to look for the mainstream customer distribution, to have more information about customer segment 

In [ ]:
premium_lifetage_customer_df = chips_df.groupby(['PREMIUM_CUSTOMER', 'LIFESTAGE'])['PROD_QTY'].sum()
premium_lifetage_customer_df = premium_lifetage_customer_df.to_frame().reset_index()
premium_lifetage_customer_df

Following observations can be made: 
1. In total, older couples/singles spent more on the chips as compared to the remaining customer segement. 
2. Mainstream customer spent more on chips as compared to budget and premium customers. 
3. On further analysis, one can see that in 

            budget customer segment :  older families
    
            mainstream customer segment : young singles/couples
    
            premium customer segment : older singles/couples
    
   are the people who make up the major share of chips customer in their segment. 

In [ ]:
budget_df = premium_lifetage_customer_df[premium_lifetage_customer_df["PREMIUM_CUSTOMER"] == 'Budget']
budget_df

In [ ]:
fig3 = px.bar(data_frame = budget_df, x = 'LIFESTAGE', y = 'PROD_QTY', orientation = 'v', title = 'Analysis for Budget customer segment')
fig3.show()

In [ ]:
main_df = premium_lifetage_customer_df[premium_lifetage_customer_df["PREMIUM_CUSTOMER"] == 'Mainstream']
main_df

In [ ]:
fig4 = px.bar(data_frame = main_df, x = 'LIFESTAGE', y = 'PROD_QTY', orientation = 'v', title = 'Analysis for Mainstream customer segment')
fig4.show()

In [ ]:
pre_df = premium_lifetage_customer_df[premium_lifetage_customer_df["PREMIUM_CUSTOMER"] == 'Premium']
pre_df

In [ ]:
fig5 = px.bar(data_frame = pre_df, x = 'LIFESTAGE', y = 'PROD_QTY', orientation = 'v', title = 'Analysis for Premium customer segment')
fig5.show()